In [1]:
def task1():
    import tensorflow as tf
    from hops import tensorboard
    from hops import hdfs
    
    from tensorflow.examples.tutorials.mnist import input_data
    
    fashion_mnist = input_data.read_data_sets('input/data', one_hot=True)
    
    # Hardcoded params
    num_ch = 1
    num_classes = 10
    image_height = image_width = 28
    
    # 1. Define variables and placeholders
    X = tf.placeholder(tf.float32, shape=[None, image_height, image_width, num_ch])
    Y_ = tf.placeholder(tf.float32, shape=[None, 10])
    
    W = tf.Variable(tf.zeros([image_height * image_width, num_classes]))
    b = tf.Variable(tf.zeros([num_classes]))
    
    # Flatten images into a single vector
    XX = tf.reshape(X, [-1, image_height * image_width])
    
    # 2. Define the model Y = softmax(XX*W + b)
    # Docs state that cross entropy fn below does NOT take an already-softmaxed input
    # Y = tf.nn.softmax(tf.matmul(XX, W) + b )
    Y = tf.matmul(XX, W) + b
    
    # 3a. Define the loss function (neg. log loss) and 3b.compute its mean over the trained batches
    #cross_entropy_mean = tf.reduce_mean(-tf.reduce_sum(Y_ * tf.log(Y), reduction_indices=[1]))
    cross_entropy_mean = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_))
    
    # 4. Accuracy definition
    correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Define optimizers used
    with tf.name_scope('train'):
        with tf.name_scope('gradient_descent'):
            train_step_gd = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy_mean)
        with tf.name_scope('adam_optimizer'):
            train_step_adam = tf.train.AdamOptimizer(0.005).minimize(cross_entropy_mean)

    
    # Define accuracy
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction =  tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    init =  tf.global_variables_initializer()
    sess = tf.Session()    
    
    def epochs(train, test, train_step, num_epochs=100, batch_size=100):

        sess.run(init)
        
        accuracies = []
        losses = []
        for epoch in range(num_epochs):
            
            for it in range(100):
                batch_xs, batch_ys = train.next_batch(batch_size)
                feed_dict =  {XX: batch_xs, Y_: batch_ys}
                sess.run(train_step, feed_dict)

            # Compute accuracy and loss every 100 rounds
            feed_dict = {XX: test.images, Y_: test.labels}
            acc = sess.run(accuracy, feed_dict)
            loss = sess.run(cross_entropy_mean, feed_dict)
            
            accuracies.append(acc)
            losses.append(loss)
        return (accuracies, losses)


    acc, loss = epochs(fashion_mnist.train, fashion_mnist.test, train_step_adam)
    
    print("Loss: {}".format(loss))
    print("Accuracy: {}".format(acc))
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    logdir = tensorboard.logdir()
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(logdir + '/train',
                                          sess.graph)
    test_writer = tf.summary.FileWriter(logdir + '/test')
    
    train_writer.close()
    test_writer.close()


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1095,application_1511276242554_0429,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from hops import tflauncher
from hops import util

# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()

tb_hdfs_dir = tflauncher.launch(spark, task1)
#tensorboard.visualize(spark, tb_hdfs_dir)


Finished TensorFlow job 

Make sure to check /Logs/TensorFlow/application_1511276242554_0429/runId.0 for logfile and TensorBoard logdir